In [1]:
import os
import random
import shutil

import matplotlib.pyplot as plt
import cv2
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D
from keras.models import load_model

from tensorflow.keras.optimizers import Adam

import json

In [2]:
cd D:\SEMESTER 7\MachineLearning\Praktikum\Modul 5\brain-tumor

D:\SEMESTER 7\MachineLearning\Praktikum\Modul 5\brain-tumor


In [3]:
ls

 Volume in drive D is Data
 Volume Serial Number is 7453-53B1

 Directory of D:\SEMESTER 7\MachineLearning\Praktikum\Modul 5\brain-tumor

17/12/2021  14:28    <DIR>          .
17/12/2021  14:28    <DIR>          ..
17/12/2021  14:27    <DIR>          1
17/12/2021  14:28    <DIR>          2
17/12/2021  14:28    <DIR>          3
               0 File(s)              0 bytes
               5 Dir(s)  33.255.096.320 bytes free


In [6]:
base_dir = 'D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor'

men_dir = os.path.join(base_dir, '1')
gli_dir = os.path.join(base_dir, '2')
pit_dir = os.path.join(base_dir, '3')

In [7]:
men_data_count = len(os.listdir(men_dir))
gli_data_count = len(os.listdir(gli_dir))
pit_data_count = len(os.listdir(pit_dir))

print(f"Jumlah Meningioma: {men_data_count}\nJumlah Glioma: {gli_data_count}\nJumlah Pituitary: {pit_data_count}")

Jumlah Meningioma: 708
Jumlah Glioma: 1426
Jumlah Pituitary: 930


In [8]:
glioma_img = os.path.join(base_dir, '2_RES')
gli_img_names = os.listdir(gli_dir)
gli_img_names = random.sample(gli_img_names, men_data_count)
count = 0
for gli_img in gli_img_names:
    gli_img_dir = os.path.join(gli_dir, gli_img)
    os.makedirs(os.path.dirname(glioma_img), exist_ok=True)
    shutil.copy(gli_img_dir, glioma_img)
    count += 1
print(f"Data tersalin: {count}")

Data tersalin: 708


In [13]:
pituitary_img = os.path.join(base_dir, '3_RES')
pit_img_names = os.listdir(pit_dir)
pit_img_names = random.sample(pit_img_names, men_data_count)
count = 0
for pit_img in pit_img_names:
    pit_img_dir = os.path.join(pit_dir, pit_img)
    os.makedirs(os.path.dirname(pituitary_img), exist_ok=True)
    shutil.copy(pit_img_dir, pituitary_img)
    count += 1
print(f"Data tersalin: {count}")

Data tersalin: 708


In [14]:
res_gli_dir = os.path.join(base_dir, '2_RES')
res_pit_dir = os.path.join(base_dir, '3_RES')
res_gli_data = len(os.listdir(res_gli_dir))
res_pit_data = len(os.listdir(res_pit_dir))
print(f"Jumlah Meningioma: {men_data_count}\nJumlah Glioma: {res_gli_data}\nJumlah Pituitary: {res_pit_data}")

Jumlah Meningioma: 708
Jumlah Glioma: 708
Jumlah Pituitary: 708


In [15]:
import splitfolders
base_dir="D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor"
data_dir="D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor-splitted"
splitfolders.ratio(base_dir, output=data_dir, seed=1337, ratio=(.8 ,0.19 ,0.01),group_prefix=None)

Copying files: 4480 files [00:19, 230.25 files/s]


In [16]:
base_dir = 'D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor-splitted'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

train_men_dir = os.path.join(train_dir, '1')
train_gli_dir = os.path.join(train_dir, '2_RES')
train_pit_dir = os.path.join(train_dir, '3_RES')

validation_men_dir = os.path.join(validation_dir, '1')
validation_gli_dir = os.path.join(validation_dir, '2_RES')
validation_pit_dir = os.path.join(validation_dir, '3_RES')

test_men_dir = os.path.join(test_dir, '1')
test_gli_dir = os.path.join(test_dir, '2_RES')
test_pit_dir = os.path.join(test_dir, '3_RES')

In [18]:
print("Total Training MENINGIOMA:", len(os.listdir(train_men_dir)))
print("Total Training GLIOMA:", len(os.listdir(train_gli_dir)))
print("Total Training PITUITARY:", len(os.listdir(train_pit_dir)))
print('-'*30)
print("Total validation MENINGIOMA:", len(os.listdir(validation_men_dir)))
print("Total validation GLIOMA:", len(os.listdir(validation_gli_dir)))
print("Total validation PITUITARY:", len(os.listdir(validation_pit_dir)))
print('-'*30)
print("Total Testing MENINGIOMA:", len(os.listdir(test_men_dir)))
print("Total Testing GLIOMA:", len(os.listdir(test_gli_dir)))
print("Total Testing PITUITARY:", len(os.listdir(test_pit_dir)))

Total Training MENINGIOMA: 566
Total Training GLIOMA: 566
Total Training PITUITARY: 566
------------------------------
Total validation MENINGIOMA: 134
Total validation GLIOMA: 134
Total validation PITUITARY: 134
------------------------------
Total Testing MENINGIOMA: 8
Total Testing GLIOMA: 8
Total Testing PITUITARY: 8


In [19]:
TRAINING_DIR = "D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor-splitted/train"
VALIDATION_DIR = "D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor-splitted/val"
TEST_DIR = "D:/SEMESTER 7/MachineLearning/Praktikum/Modul 5/brain-tumor-splitted/test"

datagen = ImageDataGenerator(rescale= 0)

train_gen = datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    shuffle=False,
    class_mode='categorical',
    batch_size = 32
)

val_gen = datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    shuffle=False,
    class_mode='categorical',
    batch_size = 32
)

test_gen = datagen.flow_from_directory(
    TEST_DIR,
    target_size=(150,150),
    shuffle=False,
    class_mode='categorical',
    batch_size = 32
)

Found 1698 images belonging to 3 classes.
Found 402 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [20]:
vgg19_base = VGG19(
    include_top=False,
    input_shape=(150, 150, 3)
    )

for layer in vgg19_base.layers:
    layer.trainable = False
    layer._name = layer._name + str('_19')

In [30]:
input_layer = Input((150, 150, 3))

fc_layer = Flatten()(vgg19_base(input_layer))
fc_layer = Dense(128, activation='relu')(fc_layer)
fc_layer = Dropout(0.5)(fc_layer)
fc_layer = Dense(32, activation='relu')(fc_layer)
fc_layer = Dropout(0.2)(fc_layer)

output_layer = Dense(3, activation='softmax')(fc_layer)

model = Model(input_layer, output_layer)
print(model.summary())

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               1048704   
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 32)                4128      
                                                                 
 dropout_7 (Dropout)         (None, 32)                0   

In [31]:
model.compile(optimizer=Adam(learning_rate=0.001,), 
              loss='categorical_crossentropy', 
              metrics=['acc'])

In [32]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop_val_acc = EarlyStopping(monitor='val_accuracy',
                          patience=5,
                          verbose=1,
                          mode='auto'
                          )

In [33]:
H = model.fit(train_gen,
              validation_data=val_gen,
              epochs=150,
              verbose=1,
             callbacks=early_stop_val_acc)

Epoch 1/150
54/54 [==============================] - 47s 846ms/step - loss: 9.7841 - acc: 0.4452 - val_loss: 0.9803 - val_acc: 0.7189
Epoch 2/150
10/54 [====>.........................] - ETA: 20s - loss: 2.5863 - acc: 0.5344

KeyboardInterrupt: 

In [34]:
H2 = model.fit(train_gen,
              validation_data=val_gen,
              epochs=150,
              verbose=1,)

Epoch 1/150
54/54 [==============================] - 31s 580ms/step - loss: 1.2121 - acc: 0.4817 - val_loss: 0.6318 - val_acc: 0.7438
Epoch 2/150
54/54 [==============================] - 32s 589ms/step - loss: 0.9135 - acc: 0.5830 - val_loss: 0.7308 - val_acc: 0.6915
Epoch 3/150
54/54 [==============================] - 32s 589ms/step - loss: 0.8796 - acc: 0.5954 - val_loss: 0.6115 - val_acc: 0.7488
Epoch 4/150
54/54 [==============================] - 30s 560ms/step - loss: 0.8369 - acc: 0.6290 - val_loss: 0.6632 - val_acc: 0.7836
Epoch 5/150
54/54 [==============================] - 19s 355ms/step - loss: 0.7535 - acc: 0.6667 - val_loss: 0.5214 - val_acc: 0.8035
Epoch 6/150
54/54 [==============================] - 19s 359ms/step - loss: 0.7009 - acc: 0.7079 - val_loss: 0.5617 - val_acc: 0.8060
Epoch 7/150
54/54 [==============================] - 19s 360ms/step - loss: 0.7055 - acc: 0.6808 - val_loss: 0.5010 - val_acc: 0.8383
Epoch 8/150
54/54 [==============================] - 20s 364ms

Epoch 62/150
54/54 [==============================] - 21s 385ms/step - loss: 0.3647 - acc: 0.7886 - val_loss: 0.4319 - val_acc: 0.9055
Epoch 63/150
54/54 [==============================] - 21s 385ms/step - loss: 0.3984 - acc: 0.7727 - val_loss: 0.5437 - val_acc: 0.8607
Epoch 64/150
54/54 [==============================] - 21s 381ms/step - loss: 0.4009 - acc: 0.7892 - val_loss: 0.4275 - val_acc: 0.9005
Epoch 65/150
54/54 [==============================] - 21s 380ms/step - loss: 0.4123 - acc: 0.7638 - val_loss: 0.3869 - val_acc: 0.8881
Epoch 66/150
54/54 [==============================] - 21s 385ms/step - loss: 0.4055 - acc: 0.7580 - val_loss: 0.4179 - val_acc: 0.8831
Epoch 67/150
54/54 [==============================] - 21s 383ms/step - loss: 0.3700 - acc: 0.7768 - val_loss: 0.4125 - val_acc: 0.8806
Epoch 68/150
54/54 [==============================] - 21s 385ms/step - loss: 0.3615 - acc: 0.7750 - val_loss: 0.3989 - val_acc: 0.8905
Epoch 69/150
54/54 [==============================] - 2

54/54 [==============================] - 21s 382ms/step - loss: 0.3434 - acc: 0.7756 - val_loss: 0.4408 - val_acc: 0.9055
Epoch 123/150
54/54 [==============================] - 21s 382ms/step - loss: 0.3338 - acc: 0.7721 - val_loss: 0.4505 - val_acc: 0.8881
Epoch 124/150
54/54 [==============================] - 21s 384ms/step - loss: 0.3445 - acc: 0.7856 - val_loss: 0.3940 - val_acc: 0.9030
Epoch 125/150
54/54 [==============================] - 21s 384ms/step - loss: 0.3322 - acc: 0.7880 - val_loss: 0.4487 - val_acc: 0.8731
Epoch 126/150
54/54 [==============================] - 20s 378ms/step - loss: 0.3117 - acc: 0.8180 - val_loss: 0.4110 - val_acc: 0.8582
Epoch 127/150
54/54 [==============================] - 21s 385ms/step - loss: 0.3132 - acc: 0.8068 - val_loss: 0.5121 - val_acc: 0.8955
Epoch 128/150
54/54 [==============================] - 21s 384ms/step - loss: 0.3327 - acc: 0.8045 - val_loss: 0.4641 - val_acc: 0.8682
Epoch 129/150
54/54 [==============================] - 21s 383

In [35]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
save_tflite = 'D:\SEMESTER 7\MachineLearning\Praktikum\Modul 5\model_kelompok.tflite'
with open(save_tflite, 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DKID~1\AppData\Local\Temp\tmptl0w6fua\assets
